## 2752. Customers with Maximum Number of Transactions on Consecutive Days
```
Table: Transactions
+------------------+------+
| Column Name      | Type |
+------------------+------+
| transaction_id   | int  |
| customer_id      | int  |
| transaction_date | date |
| amount           | int  |
+------------------+------+
transaction_id is the column with unique values of this table.
Each row contains information about transactions that includes unique (customer_id, transaction_date) along with the corresponding customer_id and amount.   
```
### Task: Write a solution to find all customer_id who made the maximum number of transactions on consecutive days.
```
Return all customer_id with the maximum number of consecutive transactions. Order the result table by customer_id in ascending order.

The result format is in the following example. 

Example 1:

Input: 
Transactions table:
+----------------+-------------+------------------+--------+
| transaction_id | customer_id | transaction_date | amount |
+----------------+-------------+------------------+--------+
| 1              | 101         | 2023-05-01       | 100    |
| 2              | 101         | 2023-05-02       | 150    |
| 3              | 101         | 2023-05-03       | 200    |
| 4              | 102         | 2023-05-01       | 50     |
| 5              | 102         | 2023-05-03       | 100    |
| 6              | 102         | 2023-05-04       | 200    |
| 7              | 105         | 2023-05-01       | 100    |
| 8              | 105         | 2023-05-02       | 150    |
| 9              | 105         | 2023-05-03       | 200    |
+----------------+-------------+------------------+--------+

Output: 
+-------------+
| customer_id | 
+-------------+
| 101         | 
| 105         | 
+-------------+

Explanation: 
- customer_id 101 has a total of 3 transactions, and all of them are consecutive.
- customer_id 102 has a total of 3 transactions, but only 2 of them are consecutive. 
- customer_id 105 has a total of 3 transactions, and all of them are consecutive.

In total, the highest number of consecutive transactions is 3, achieved by customer_id 101 and 105. The customer_id are sorted in ascending order.
```

### Solution #1: 22/23 testcases passed
```
# Write your MySQL query statement below
WITH set1 AS (
    SELECT
        t1.customer_id,
        t1.transaction_date AS start_date,
        t2.transaction_date AS end_date,
        ROW_NUMBER() OVER (PARTITION BY t1.customer_id ORDER BY t1.transaction_date) AS rn
    FROM
        Transactions AS t1
    INNER JOIN
        Transactions AS t2
             ON t2.customer_id = t1.customer_id
            AND DATEDIFF(t2.transaction_date, t1.transaction_date) = 1
),
set2 AS (
    SELECT
        customer_id,
        start_date,
        end_date,
        DATE_SUB(start_date, INTERVAL rn DAY) AS grp_date
    FROM
        set1
),
set3 AS (
    SELECT
        customer_id,
        -- grp_date,
        DATEDIFF(MAX(end_date), MIN(start_date)) + 1 AS cnt
    FROM set2
    GROUP BY customer_id, grp_date
)
SELECT customer_id
FROM set3 
WHERE cnt = (SELECT MAX(cnt) FROM set3)
ORDER BY 1;
```